In [1]:
import warnings
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.appName("Balance Calculator").getOrCreate()

24/07/03 10:21:59 WARN Utils: Your hostname, Bisheshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.13.163.225 instead (on interface en0)
24/07/03 10:21:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/03 10:21:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/03 10:22:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/03 10:22:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [7]:
df = spark.read.csv('../datasets/transaction_summary.csv', inferSchema=True, header=True)
df.printSchema()
df.show(5)

root
 |-- Account No: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- TRANSACTION DETAILS: string (nullable = true)
 |-- CHQ.NO.: integer (nullable = true)
 |-- VALUE DATE: string (nullable = true)
 |-- WITHDRAWAL AMT: double (nullable = true)
 |-- DEPOSIT AMT: double (nullable = true)
 |-- BALANCE AMT: double (nullable = true)

+-------------+---------+--------------------+-------+----------+--------------+-----------+-----------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|WITHDRAWAL AMT|DEPOSIT AMT|BALANCE AMT|
+-------------+---------+--------------------+-------+----------+--------------+-----------+-----------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|   NULL| 29-Jun-17|          NULL|  1000000.0|  1000000.0|
|409000611074'| 5-Jul-17|TRF FROM  Indiafo...|   NULL|  5-Jul-17|          NULL|  1000000.0|  2000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|   NULL| 18-Jul-17|          NULL|   500000.0|  2500000.0|
|409000611074'| 1-Aug

In [8]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

In [13]:
df = df.withColumn('WITHDRAWAL AMT', regexp_replace('WITHDRAWAL AMT', ',', ''))
df = df.withColumn('DEPOSIT AMT', regexp_replace('DEPOSIT AMT', ',', ''))
df = df.withColumn('WITHDRAWAL AMT', col('WITHDRAWAL AMT').cast(DoubleType()))
df = df.withColumn('DEPOSIT AMT', col('DEPOSIT AMT').cast(DoubleType()))
df.show(5)

+-------------+---------+--------------------+-------+----------+--------------+-----------+-----------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|WITHDRAWAL AMT|DEPOSIT AMT|BALANCE AMT|
+-------------+---------+--------------------+-------+----------+--------------+-----------+-----------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|   NULL| 29-Jun-17|          NULL|  1000000.0|  1000000.0|
|409000611074'| 5-Jul-17|TRF FROM  Indiafo...|   NULL|  5-Jul-17|          NULL|  1000000.0|  2000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|   NULL| 18-Jul-17|          NULL|   500000.0|  2500000.0|
|409000611074'| 1-Aug-17|TRF FRM  Indiafor...|   NULL|  1-Aug-17|          NULL|  3000000.0|  5500000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|   NULL| 16-Aug-17|          NULL|   500000.0|  6000000.0|
+-------------+---------+--------------------+-------+----------+--------------+-----------+-----------+
only showing top 5 rows



In [18]:
df = df.withColumnRenamed('WITHDRAWAL AMT','wit')
df = df.withColumnRenamed('DEPOSIT AMT','dep')

In [19]:
df1 = df.fillna(0)

In [20]:
df1.show(5)

+-------------+---------+--------------------+-------+----------+---+---------+-----------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|wit|      dep|BALANCE AMT|
+-------------+---------+--------------------+-------+----------+---+---------+-----------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|      0| 29-Jun-17|0.0|1000000.0|  1000000.0|
|409000611074'| 5-Jul-17|TRF FROM  Indiafo...|      0|  5-Jul-17|0.0|1000000.0|  2000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|      0| 18-Jul-17|0.0| 500000.0|  2500000.0|
|409000611074'| 1-Aug-17|TRF FRM  Indiafor...|      0|  1-Aug-17|0.0|3000000.0|  5500000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|0.0| 500000.0|  6000000.0|
+-------------+---------+--------------------+-------+----------+---+---------+-----------+
only showing top 5 rows



In [25]:
df2 = df1.withColumn('Dep-Wit',df1.dep - df1.wit)
df2.show(5)

+-------------+---------+--------------------+-------+----------+---+---------+-----------+---------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|wit|      dep|BALANCE AMT|  Dep-Wit|
+-------------+---------+--------------------+-------+----------+---+---------+-----------+---------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|      0| 29-Jun-17|0.0|1000000.0|  1000000.0|1000000.0|
|409000611074'| 5-Jul-17|TRF FROM  Indiafo...|      0|  5-Jul-17|0.0|1000000.0|  2000000.0|1000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|      0| 18-Jul-17|0.0| 500000.0|  2500000.0| 500000.0|
|409000611074'| 1-Aug-17|TRF FRM  Indiafor...|      0|  1-Aug-17|0.0|3000000.0|  5500000.0|3000000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|0.0| 500000.0|  6000000.0| 500000.0|
+-------------+---------+--------------------+-------+----------+---+---------+-----------+---------+
only showing top 5 rows



In [22]:
# df2 = df2.withColumn("Date", to_date(col("DATE"), "dd-MM-yy"))

In [26]:
df2.show()

+-------------+---------+--------------------+-------+----------+--------+---------+-----------+---------+
|   Account No|     DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|     wit|      dep|BALANCE AMT|  Dep-Wit|
+-------------+---------+--------------------+-------+----------+--------+---------+-----------+---------+
|409000611074'|29-Jun-17|TRF FROM  Indiafo...|      0| 29-Jun-17|     0.0|1000000.0|  1000000.0|1000000.0|
|409000611074'| 5-Jul-17|TRF FROM  Indiafo...|      0|  5-Jul-17|     0.0|1000000.0|  2000000.0|1000000.0|
|409000611074'|18-Jul-17|FDRL/INTERNAL FUN...|      0| 18-Jul-17|     0.0| 500000.0|  2500000.0| 500000.0|
|409000611074'| 1-Aug-17|TRF FRM  Indiafor...|      0|  1-Aug-17|     0.0|3000000.0|  5500000.0|3000000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|     0.0| 500000.0|  6000000.0| 500000.0|
|409000611074'|16-Aug-17|FDRL/INTERNAL FUN...|      0| 16-Aug-17|     0.0| 500000.0|  6500000.0| 500000.0|
|409000611074'|16-Aug-17|FDRL/INTERNA

In [27]:
from pyspark.sql.window import Window

In [28]:
window = Window.partitionBy('Account No').orderBy("Date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df2 = df2.withColumn("CumulativeSum", sum("Dep-Wit").over(window))

In [29]:
df2.show(500)

+----------+--------+--------------------+-------+----------+------------+-------------+----------------+-------------+--------------------+
|Account No|    DATE| TRANSACTION DETAILS|CHQ.NO.|VALUE DATE|         wit|          dep|     BALANCE AMT|      Dep-Wit|       CumulativeSum|
+----------+--------+--------------------+-------+----------+------------+-------------+----------------+-------------+--------------------+
|  1196428'|1-Apr-17|IB:TPFT0000356586...|      0|  1-Apr-17|         0.0|     330000.0|-1.68591617279E9|     330000.0|            330000.0|
|  1196428'|1-Aug-15|INTERNAL FUND TRA...|      0|  1-Aug-15|   5000000.0|          0.0| -1.5802964826E9|   -5000000.0|          -4670000.0|
|  1196428'|1-Aug-15|   CASHDEP/SILVASSA/|      0|  1-Aug-15|         0.0|      60500.0| -1.5802359826E9|      60500.0|          -4609500.0|
|  1196428'|1-Aug-15|  CASHDEP/NEW-DELHI/|      0|  1-Aug-15|         0.0|     100000.0| -1.5801359826E9|     100000.0|          -4509500.0|
|  1196428'|1

In [30]:
df2.write.csv('Balance_Calc.csv', header=True)

In [31]:
spark.stop()